In [1]:
import pandas as pd
from sklearn import preprocessing
import pickle

Nous allons mettre en place une fonction permettant de detecter un vrai/faux billet de banque 
pour que cela soit possible il faux que le dataset contienne les colonnes suivantes : diagonal height_left
height_right margin_low margin_up et length

In [2]:
def detectBankNote(fileName):
    
    
    banknoteDataframe = pd.read_csv("../../lab/origin_values/" + fileName, parse_dates=True) 
    
    ## on copy le df example car nous allons lui enlever 
    banknoteSave = banknoteDataframe.copy()

    ## on part du principe que le dataset contient le 
    if 'id' in banknoteSave.columns:
        banknoteSave = banknoteSave.loc[:, banknoteSave.columns != 'id']


    # reduction du dataset et standardisation lutte contre les écarts d'échelle
    scaleDf = preprocessing.StandardScaler().fit(banknoteSave)
    banknoteScaled = scaleDf.transform(banknoteSave)


    # recupération du model de regression afin de faire la déduction

    with open('../../lab/Python_traitement/modelLR.pkl', 'rb') as file:
        getRegressionLogistic = pickle.Unpickler(file).load()

    # appilcation du dataset sur la regression logistic 
    probabilityBanknoteIsGenuine = getRegressionLogistic.predict_proba(banknoteScaled)[:, 1]

    # regroup predict 

    dataframeProbabilityBanknote = pd.Series(probabilityBanknoteIsGenuine.round(2), name="percent_proba_isgenuine")


    dataframeProbabilityBanknote.head()

    concatenationDataframe = pd.concat([banknoteDataframe, dataframeProbabilityBanknote], axis=1)


    concatenationDataframe.head()

    ## nous partons du prince du boolean ici on considère que toute les proba > à 50% sont des vrais billets 
    result = []
    for i in concatenationDataframe['percent_proba_isgenuine'] >= .5:
        if i is True :
            result.append('real_banknote')
        else :
            result.append('fake_banknote')

    concatenationDataframe['result_isgenuine'] = result   
    
    return concatenationDataframe
    


In [3]:
def countRealAndFakeBanknote(df):
    nbRealBanknote = len(df[df['result_isgenuine'] == 'real_banknote'])
    nbFakeBanknote = len(df[df['result_isgenuine'] == 'fake_banknote'])

    return 'Le dataset contient : ' + str(nbRealBanknote) + ' vrais billet  et ' + str(nbFakeBanknote) + ' faux billet'

In [4]:
## il faut passer le nom du fichier 
detectBankNote("example.csv")


/usr/local/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


,diagonal,height_left,height_right,margin_low,margin_up,length,id,percent_proba_isgenuine,result_isgenuine
0,171.76,104.01,103.54,5.21,3.30,111.42,A_1,0.22,fake_banknote
1,171.87,104.17,104.13,6.00,3.31,112.09,A_2,0.04,fake_banknote
2,172.00,104.58,104.29,4.99,3.39,111.57,A_3,0.02,fake_banknote
3,172.49,104.55,104.34,4.44,3.03,113.20,A_4,0.86,real_banknote
4,171.65,103.63,103.56,3.77,3.16,113.33,A_5,1.00,real_banknote


In [5]:
print(countRealAndFakeBanknote(detectBankNote("example.csv")))
    

Le dataset contient : 2 vrais billet  et 3 faux billet


/usr/local/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [8]:
detectBankNote("test_notes.csv")

/usr/local/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


,diagonal,height_left,height_right,margin_low,margin_up,length,id,percent_proba_isgenuine,result_isgenuine
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,0.99,real_banknote
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,0.11,fake_banknote
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,1.00,real_banknote
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,0.01,fake_banknote
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,0.06,fake_banknote


In [9]:
detectBankNote("billets_test.csv")

/usr/local/lib/python3.8/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


,diagonal,height_left,height_right,margin_low,margin_up,length,id,percent_proba_isgenuine,result_isgenuine
0,172.09,103.95,103.73,4.39,3.09,113.19,B_1,0.99,real_banknote
1,171.52,104.17,104.03,5.27,3.16,111.82,B_2,0.11,fake_banknote
2,171.78,103.80,103.75,3.81,3.24,113.39,B_3,1.00,real_banknote
3,172.02,104.08,103.99,5.57,3.30,111.10,B_4,0.01,fake_banknote
4,171.79,104.34,104.37,5.00,3.07,111.87,B_5,0.06,fake_banknote
